In [93]:
import pandas as pd
from surprise import SVD, SVDpp
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import NormalPredictor
from surprise import Reader
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.prediction_algorithms.matrix_factorization import NMF
import sys
sys.path.append("./../")
from src.utils import percentileMetric
from surprise.prediction_algorithms.co_clustering import CoClustering

In [21]:
training = pd.read_csv('../data/training.csv')

In [24]:
training.drop(columns='timestamp', inplace=True)

In [25]:
training

,user,movie,rating
0,6040,858,4
1,6040,593,5
2,6040,2384,4
3,6040,1961,4
4,6040,2019,5
...,...,...,...
799995,1875,2940,5
799996,1875,589,4
799997,1875,110,4
799998,1875,2791,3


In [102]:
training['user'][0]

6040

In [32]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(training, reader)

In [86]:
trainset, testset = train_test_split(data, test_size=.25)
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.8862


0.8862384877890321

In [92]:
pred_df = pd.DataFrame(data=predictions, columns=('user','movie','actualrating','predictedrating','details'))
percentileMetric(pred_df)

4.3868986693961105

In [36]:
algo = KNNBasic()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9333


0.933280368211493

In [75]:
algo = NMF()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.9246


0.9246419988061235

In [94]:
algo = CoClustering()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
print('RMSE:', accuracy.rmse(predictions))

pred_df = pd.DataFrame(data=predictions, columns=('user','movie','actualrating','predictedrating','details'))
print('The Snyder Metric:', percentileMetric(pred_df))

/Users/kevin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


RMSE: 0.9175
RMSE: 0.9175292663454286
The Snyder Metric: 4.344111039794608


In [95]:
algo = SVDpp()

algo.fit(trainset)
predictions = algo.test(testset)

pred_df = pd.DataFrame(data=predictions, columns=('user','movie','actualrating','predictedrating','details'))
print('The Snyder Metric:', percentileMetric(pred_df))

RMSE: 0.8694
RMSE: 0.8693759616134457
The Snyder Metric: 4.424692137669719


In [100]:
algo = SVDpp(n_factors=5, n_epochs=5)

algo.fit(trainset)
predictions = algo.test(testset)

pred_df = pd.DataFrame(data=predictions, columns=('user','movie','actualrating','predictedrating','details'))
print('The Snyder Metric:', percentileMetric(pred_df))

The Snyder Metric: 4.354602097957252


In [ ]:
for idx in full_testset.index:
    full_testset.loc[idx, 'predicted_rating'] = algo.predict(str(full_testset['user'][idx]), str(full_testset['movie'][idx]))

In [101]:
trainset[0]

TypeError: 'Trainset' object is not subscriptable